# Working with APIs


Today we'll explore the nature and surrounding concepts related to APIs.

We shall experiment and get a feel for Github's Gist API.  And finally, we'll try to implement a python script which will create a new gist from files on our computer (basically upload files to github for sharing).

## API
- **Application Programming Interface**  In the context of Web APIs, this is a service provided by a web application to allow other programs to more easily interact with the Application.  The method to retrieve or send specific information is standardized, and returned in a machine-readable format.

When we were web-scraping last week, we had to figure out the URL format we wanted, then make the request, get the HTML and parse the HTML and find the specific information we wanted.  The same could be achieved with an API, except:
- the URL and its format to retrieve information is standardized and documented
- the response is not HTML but rather JSON/XML
- We still have to parse the response to find the specific information we want, but it's all documented (and consistent!)


**Note**: the term 'API' is not exclusive to the web.  The way you interact with any piece of software when programming is said to be it's API.  This includes things like the name of functions, they arguments provided to functiosn, and the general usablity as well (in what order you should call the functions, etc).

## HTTP

Since Web APIs are also built on HTTP, we will still be using the `requests` framework.  Except now we need to know a bit more about HTTP, such as the standard HTTP verbs:

- **GET**: retrieves information from the resource
- **POST**: Sends information to the resource, with the intent to create something new
- **PUT/PATCH**: Sends information to the resource, with the intent to *update* that resource.
- **DELETE**: Removes an instance of the resource (undoes a POST).

A common acronym for this functionality is CRUD - Create, Read, Update, Delete.

## REST

- **Representational State Transfer** is a pattern/style of API design.

The basic idea is to structure your applicaiton as a series of **resources**, to which you can apply any or all of the CRUD actions described above.

For example, if I'm shopping on a website I'll have a *Cart* resource, which is *created* when I start adding things to my cart.  Its *updated* when I add more things to my cart.  Its *read* when I go to check out.  When I finalize my order and pay, rather than update the cart to say that it is finalized, we might *create* a new Order.  In this example, we have a Cart resource and an Order resource.

See [this Wikipedia Table](https://en.wikipedia.org/wiki/Representational_state_transfer#Relationship_between_URL_and_HTTP_methods) for some examples.

**Note**: REST is not the only architecture pattern.  There are others but at the moment REST is most popular.

## JSON
- **Javascript Object Notation** is a lightweight data format.  It's the most popular format (these days) for working with web APIs.  It's very similar to a python dictionary, with keys and values.  Very often you will see lists and dictionaries nested inside one another with JSON.

JSON will sometimes use a different terminology or notation than python to refer to the types of information it contains.

python | json
-------|------
dictionary | object
list | array
True | true
False | false
None | null

To convert our data to and from json, we will use the builtin [json](http://docs.python-guide.org/en/latest/scenarios/json/) module.

The other option is XML, which more verbose and fading in popularity.  HTML is a type of XML.  You can see examples of both [here](http://json.org/example.html)

## Gist API

Today we'll explore working with APIs through [Github's Gist API](https://developer.github.com/v3/gists/).  There are many, many APIs out there.  I chose this one because:
- No authentication required (keeps things simple for now)
- We can do all the CRUD operations with it
- Well documented
- FREE!

A note: Github follows another architectural style called HATEOS.  You'll notice a lot of URLs in the response.  This is not as common as REST.  In most APIs you'll get the value back (i.e. `'username': 'hassanshamim'`) rather than a link to that value (i.e. `'username': 'https://github.com/user/hassanshamim'`).  Just something to be aware of.

## Getting Started

Jupyter notebook is a great place to start playing around with the API to get a feel for how to format our requests, and how to get the information we want back out.

Once we start building our script, a proper text editor is the best place to do that.

In [ ]:
import requests
import json

In [ ]:
# We'll be using these URLs a lot, so lets save the first part.  Then, we only have to worry about the resource we want.
API_BASE_URL = 'https://api.github.com/'
GIST_BASE_URL = API_BASE_URL + 'gists'

In [ ]:
# Let's do the simplest http verb first - GET
public_gists_url = GIST_BASE_URL + '/public'
response = requests.get(public_gists_url)
response.text

In [ ]:
result = json.loads(response.text) # convert to a python object - in this case a list of dictionaries
result[0] # let's look at the first one

In [ ]:
resp2 = requests.get(GIST_BASE_URL + '/starred')
resp2.text

In [ ]:
resp2.json()

In [ ]:
# Get a single gist
my_gist_id = '32ad548cf3c5779e6132ea0b1fc1a78d'
url = GIST_BASE_URL + '/' +  my_gist_id
resp3 = requests.get(url)

In [ ]:
resp3.text

In [ ]:
result3 = json.loads(resp3.text)
result3

In [ ]:
# Explore this gists history.  Look at a specific version.

In [ ]:
# Create a gist - https://developer.github.com/v3/gists/#create-a-gist
# We could just build one whole object, but I like doing it in steps.
# I think for exploration purposes it's more clear.
example_data = {}
example_data['description'] = 'test gist from api'
example_data['public'] = False
files = {}
# Each 'file' is a dictionary with the format {'file_name_here': {'contents': 'file_contents_here}}
files['example_file_name.txt'] = {'content': 'example_file_data'}
example_data['files'] = files
example_data

In [ ]:
json.dumps(example_data)

In [ ]:
response = requests.post(GIST_BASE_URL, data=json.dumps(example_data))

In [ ]:
result = json.loads(response.text)
result

In [ ]:
last_gist_url = GIST_BASE_URL + '/' + result['id']
last_gist_url

In [ ]:
requests.delete(last_gist_url)

## API wrappers

So far we've been interacting with the API 'manually' by typing in the specific URLS we want and using requests.

Often you'll see an API wrapper available that abstracts (aka hides) the implementation details (url to follow, format of requests, etc) and just lets you interact with the API itself.  These wrappers are sometimes provided by the owners of the API itself, and sometimes created by other users.

One good example of such is [Twilio](https://www.twilio.com/docs/api/rest/sending-messages), which includes examples of the API wrapper in the API docs itself.  Github has a few official API wrappers (or clients), but none are in python.  You can find unofficial python wrappers [here](https://developer.github.com/libraries/):

## Authorization

To use the API as a user, first generate an auth token [here](https://github.com/settings/tokens)

Then, include that auth token in the header of your requests like so:

**Note**: Headers are separate from the body, or *data* you provide requests.  All HTTP verbs allow headers, whereas GET and DELETE typically do not have a body.

In [ ]:
# Example of request without auth
auth_required_url = GIST_BASE_URL + '/starred'
no_auth_response = requests.get(auth_required_url)
no_auth_response.json()

In [ ]:
# Auth Setup
my_auth_token = 'b0dac1d3637d5fe47440fc33b4d975927dcffc08' # replace with your token
AUTH_HEADER = {'Authorization': 'token ' + my_auth_token}

In [ ]:
#Example request with authorization:
auth_response = requests.get(auth_required_url, headers=AUTH_HEADER)

In [ ]:
auth_response.json()

### Tools Used
- [requests](http://docs.python-requests.org/en/master/)
- Python's [json library](https://docs.python.org/3/library/json.html)
- [Github's Gist API](https://developer.github.com/v3/gists/)